In [1]:
from urllib.request import urlopen  
from tqdm import tqdm
from lxml import html
from datetime import datetime
from collections import Counter
import json
import requests
import time
import re
# from screeninfo import get_monitors
import pandas as pd
import random
import os
import ast

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

/var/folders/wl/yn_f16552qvcz999s7pwv5sm0000gn/T/ipykernel_2377/2830196434.py:11: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# US

In [2]:
# # followers_US = pd.read_csv('Influencers_US_final.csv')
# # followers_US.head()

# supplementary_folder = "Supplementary Materials"
# followers_file = os.path.join(supplementary_folder, "followers_US.txt")

# followers_US = []
# with open(followers_file, 'r') as f:
#     for line in f:
#         if "@" in line:
#             follower = line.strip()[1:]
#             print(follower)
#             followers_US.append(follower)
# f.close()

# followers_US


In [3]:
#replace as needed
folder_name = "Supplementary Materials"
followers_path = os.path.join(folder_name, "followers_US.txt")
completed_path = os.path.join(folder_name, "completed_accounts")

followers_US = []
# with open(r'''X\followers_US.txt''', 'r') as f: # PC
with open(followers_path,'r') as f: #List of accounts from raw followers list
    for line in f:
        if "@" in line:
            follower = line.strip()[1:]
            if follower not in followers_US:
                # print(follower)
                followers_US.append(follower)
f.close()

with open(completed_path, 'r') as f: #Get list of accounts that are completed
    for line in f:
        if line.startswith('total'):
            completed_list = ast.literal_eval(line[8:])

#Filter out the ones that are completed
followers_US_set = set(followers_US)
followers_US = [f for f in followers_US_set if f not in completed_list]

print(followers_US)

print(f"Number of followers to process: {len(followers_US)}")

['briebriejoy', 'HouseJudiciary', 'LPNational', 'beinlibertarian', 'theintercept', 'nbcnews', 'NPR', 'TheAtlantic', 'gmail', 'AP', 'BadFaithPod', 'BreitbartNews', 'foxnews', 'washingtonpost', 'lpnational']
Number of followers to process: 15


In [4]:
# followers_US_list = followers_US.Influencer_Xhandle.tolist()

# count = Counter(followers_US)

# followers_US = []

# for key, value in count.items():
#     if value > 2:
#         followers_US.append(key)
        
# followers_US

In [5]:
tweets_dict_US = {}

# Function to check if the Fetch More button is present
def is_element_present(driver, xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)
        return element.is_displayed()
    except NoSuchElementException:
        return False

In [6]:
driver = webdriver.Chrome()
driver.get('https://twitter.com/i/flow/login')
wait = WebDriverWait(driver, 10)


The chromedriver version (128.0.6613.137) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (129.0.6668.70); currently, chromedriver 129.0.6668.70 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


In [7]:
### Log in

# Username
username = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'input[autocomplete=username]'))
)
username.send_keys("rigautam@udel.edu")
time.sleep(3)
login_button = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '[role=button].r-13qz1uu'))
)
login_button.click()
time.sleep(3)

try: 
    # Password
    password = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[type=password]'))
    )
    password.send_keys("Sus1234!")
    time.sleep(3)
    login_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testid*=Login_Button]'))
    )
    login_button.click()
    time.sleep(3)
    
except: # if we get an error
    username = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=ocfEnterTextTextInput]"))
    )
    username.send_keys("RichaGotem")
    time.sleep(3)
    
    next_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=ocfEnterTextNextButton]"))
    )
    next_button.click()
    time.sleep(3)
    
    password = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[name*=password]"))
    )
    password.send_keys("Sus1234!")
    time.sleep(3)
    
    login_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=LoginForm_Login_Button]"))
    )
    login_button.click()
    time.sleep(3)
    

In [8]:
count = 1

keyword1 = "(Gaza* OR Israel* OR (West OR AND OR Bank) OR (Tel OR AND OR Aviv) OR (Tel OR AND OR Aviv-Yafo) OR Bedouin* OR Palestin* OR Middle OR East* OR Middle OR Eastern OR Jew* OR rabbi OR Muslim* OR Islam* OR Jihad OR Antisemiti* OR Zionis* OR IDF)"
keyword2 = "(IOF OR Hamas OR Massacre OR Genocide OR Ceasefire OR Terroris* OR Netanyahu OR Histadrut OR Haniyeh OR Yahya OR Sinwar OR Fatah OR Mohammed OR Deif)"
hashtags = "(# OR #IsraelPalestineWar OR #IsraelGazaWar OR #Gazabombing OR #CeasefireNOW OR #FreePalestine OR #StrikeForGaza OR #Gazagenocide)"
until = "2024-09-22"
since = "2023-10-07"

unprocessed = []

for profile in tqdm(followers_US):
    if count%5 == 0:
        print("5 Accounts passed, sleeping for 5 minutes.")
        time.sleep(300)
    
    if profile not in tweets_dict_US.keys():
        print(profile)

        tweets_dict_US[profile] = []

        ### Begin scrape
        time.sleep(3)

        # Search
        search_button = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search and explore"]'))
        )
        search_button.click()
        time.sleep(3)

        search_bar = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search"]'))
        )
        search_bar.click()
        time.sleep(3)

        for i in range(2): 
            search_query = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search query"]')))

            # Advanced search query
            if i == 0:
                search_query.send_keys(f'{keyword1} {hashtags} (from:{profile}) min_faves:1000 until:{until} since:{since} -filter:links -filter:replies')
            if i == 1:
                search_query.send_keys([Keys.BACKSPACE]*1000)
                search_query.send_keys(f'{keyword2} {hashtags} (from:{profile}) min_faves:1000 until:{until} since:{since} -filter:links -filter:replies')

            search_query.send_keys(Keys.RETURN)
            time.sleep(6)
            
            # Scroll until you cannot scroll anymore
            reached_page_end = False
            last_height = driver.execute_script("return document.body.scrollHeight")

            retry_xpath = "//button[@role='button' and @type='button']//span[text()='Retry']"
            # random.randrange(10, 15)
            if (is_element_present(driver, retry_xpath)):
                try:
                    print(f'{profile} - {count} - Retry button appeared.')
                    unprocessed.append(profile)
                    for i in range(5):
                        search_button = wait.until(EC.element_to_be_clickable((By.XPATH, retry_xpath)))
                        search_button.click()
                    time.sleep(10) 
                except:
                    print(f"{profile} - Failed to click Retry")

            while not reached_page_end:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")   
                time.sleep(3)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if last_height == new_height:
                    reached_page_end = True
                else:
                    last_height = new_height
            try:
                # Select the home timeline div
                search_results = driver.find_element(By.CSS_SELECTOR, '[aria-label="Timeline: Search timeline"]')
                # Just get all the HTML, will parse later
                tweets_listed = search_results.get_attribute('innerHTML')
                time.sleep(3)

                # Append to dict
                tweets_dict_US[profile].append(tweets_listed)

            except:
                pass

    count+=1

print("Unprocessed accounts for this run:")
for profile in unprocessed:
    print(f"@{profile}\n")


  0%|          | 0/15 [00:00<?, ?it/s]

briebriejoy


  7%|▋         | 1/15 [01:08<16:05, 68.98s/it]

HouseJudiciary


 13%|█▎        | 2/15 [02:19<15:10, 70.06s/it]

...

washingtonpost


 93%|█████████▎| 14/15 [26:50<01:34, 94.04s/it] 

5 Accounts passed, sleeping for 5 minutes.
lpnational


100%|██████████| 15/15 [33:11<00:00, 132.79s/it]

Unprocessed accounts for this run:
@nbcnews

@nbcnews

@NPR

@NPR

@TheAtlantic

@TheAtlantic



In [9]:
driver.quit()

In [10]:
tweets_dict_US.keys()

dict_keys(['briebriejoy', 'HouseJudiciary', 'LPNational', 'beinlibertarian', 'theintercept', 'nbcnews', 'NPR', 'TheAtlantic', 'gmail', 'AP', 'BadFaithPod', 'BreitbartNews', 'foxnews', 'washingtonpost', 'lpnational'])

In [11]:
tweets_dict_US_cleaned = {}
    
for key, value in tweets_dict_US.items():
    if key not in tweets_dict_US_cleaned.keys():
        tweets_dict_US_cleaned[key] = []
        for v in value:
            soup = BeautifulSoup(v, 'html.parser')
            tweets = soup.find_all(attrs={"data-testid": "tweetText"})
            for tweet in tweets:
                tweet_text = tweet.get_text(strip=True)
                tweets_dict_US_cleaned[key].append(tweet_text)

tweets_dict_US_cleaned.keys()

dict_keys(['briebriejoy', 'HouseJudiciary', 'LPNational', 'beinlibertarian', 'theintercept', 'nbcnews', 'NPR', 'TheAtlantic', 'gmail', 'AP', 'BadFaithPod', 'BreitbartNews', 'foxnews', 'washingtonpost', 'lpnational'])

In [12]:
raw_data_folder = "Raw Data"
raw_json = os.path.join(raw_data_folder, f"US_tweets_{until}_{since}.json")
cleaned_json = os.path.join(raw_data_folder, f"US_tweets_cleaned_{until}_{since}.json")

In [13]:
with open(raw_json, 'w') as file:
    json.dump(tweets_dict_US, file)

In [14]:
with open(cleaned_json, 'w') as file:
    json.dump(tweets_dict_US_cleaned, file)